In [1]:
import requests
#from bs4 import BeautifulSoup
import numpy as np
import pandas as pd # library for data analsysis

#to fix the certificate issue
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

canada = pd.read_html(url)

canada_df = canada[0]

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
canada_df = canada_df.drop(canada_df[canada_df['Borough'] == 'Not assigned'].index)

#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, 
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 
#in the above table.
canada_df['Postal Code'] = canada_df['Postal Code'].astype(str)
canada_df = canada_df.groupby(level = 0, sort=False).agg( ','.join)

#adjust row numbers
canada_df.index = range(len(canada_df))

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for i in range(len(canada_df)):
 if (canada_df.iloc[[i]]['Neighbourhood'] == 'Not assigned').bool():
    canada_df.iloc[[i]]['Neighbourhood'] = canada_df.iloc[[i]]['Borough']  


In [2]:
#add longitude and latitude columns and initialize with 0
canada_df.insert(3,'Latitude', value = 0)
canada_df.insert(4,'Longitude', value = 0)


In [3]:
import html5lib

#load csv file to c 
url='http://cocl.us/Geospatial_data'
c=pd.read_csv(url)

#add latitude and longitude

for i in range(len(canada_df)):

 postal = canada_df.iloc[i]['Postal Code']
 unique_index = pd.Index(c['Postal Code'])
 ind = unique_index.get_loc(postal)
 canada_df.loc[i,'Latitude'] = c['Latitude'][ind]
 canada_df.loc[i,'Longitude'] = c['Longitude'][ind]   

In [4]:
canada_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
import folium

# create map of TORONTO using latitude and longitude values
# We start at Downtown Toronto, that is 43.654260,-79.360636

latitude = 43.654260
longitude = -79.360636

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(canada_df['Latitude'], canada_df['Longitude'], canada_df['Borough'], canada_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [7]:
canada_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
